In [2]:
app=[]

In [3]:
import csv
import pandas as pd
import math
import numpy as np
import os
import json
import warnings
warnings.filterwarnings("ignore")

def calculate_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

def num(v):
    if v == 1:
        return 'one'
    elif v == 2:
        return 'two'
    elif v == 3:
        return 'three'
    elif v == 4:
        return 'four'
    elif v == 5:
        return 'five'
    elif v == 6:
        return 'six'
    elif v == 7:
        return 'seven'
    elif v == 8:
        return 'eight'
    elif v == 9:
        return 'nine'
    

def get_direction_origin(x_a, y_a):
    direction = ""

    if x_a > 0 and y_a > 0:
        direction = "northeast"
    elif x_a > 0 and y_a < 0:
        direction = "southeast"
    elif x_a < 0 and y_a < 0:
        direction = "southwest"
    elif x_a < 0 and y_a > 0:
        direction = "northwest"
    elif x_a > 0:
        direction = "east"
    elif y_a > 0:
        direction = "north"
    elif x_a < 0:
        direction = "west"
    elif y_a < 0:
        direction = "south"

    return direction

def get_direction(x_a, y_a, x_b, y_b):
    x_diff = x_b - x_a
    y_diff = y_b - y_a
    directions = ""

    angle = math.degrees(math.atan2(y_diff, x_diff))  # Convert negative angles to positive
    if angle<0:
        angle = angle+360

    if angle>337.5 and angle<=22.5 and angle==0 and angle==360:
        directions = 'East'
    elif angle>22.5 and angle<=67.5:
        directions = 'Northeast'
    elif angle>67.5 and angle<=112.5:
        directions = 'North'
    elif angle>112.5 and angle<=157.5:
        directions = 'Northwest'
    elif angle>157.5 and angle<=202.5:
        directions = 'West'
    elif angle>202.5 and angle<=247.5:
        directions = 'Southwest'
    elif angle>247.5 and angle<=292.5:
        directions= 'South'
    elif angle>292.5 and angle<=337.5:
        directions = 'Southeast'

    return directions

def get_direction2(x_a, y_a, x_b, y_b):
    x_diff = x_b - x_a
    y_diff = y_b - y_a
    directions = ""

    angle = math.degrees(math.atan2(y_diff, x_diff))  # Convert negative angles to positive
    if angle<0:
        angle = angle+360

    if angle>337.5 and angle<=22.5 and angle==0 and angle==360:
        directions = 'West'
    elif angle>22.5 and angle<=67.5:
        directions = 'Southwest'
    elif angle>67.5 and angle<=112.5:
        directions = 'South'
    elif angle>112.5 and angle<=157.5:
        directions = 'Southeast'
    elif angle>157.5 and angle<=202.5:
        directions = 'East'
    elif angle>202.5 and angle<=247.5:
        directions = 'Northeast'
    elif angle>247.5 and angle<=292.5:
        directions= 'North'
    elif angle>292.5 and angle<=337.5:
        directions = 'Northwest'

    return directions

def is_colliding(box1, box2):

    x1_box1 = int(box1['x1'].iloc[0])
    y1_box1 = int(box1['y1'].iloc[0])
    x2_box1 = int(box1['x2'].iloc[0])
    y2_box1 = int(box1['y2'].iloc[0])

    x1_box2 = int(box2['x1'])
    y1_box2 = int(box2['y1'])
    x2_box2 = int(box2['x2'])
    y2_box2 = int(box2['y2'])

    centerx_box1 = int((x1_box1 + x2_box1)/2)
    centery_box1 = int((y1_box1 + y2_box1)/2)

    centerx_box2 = int((x1_box2 + x2_box2)/2)
    centery_box2 = int((y1_box2 + y2_box2)/2)

    center_distance = math.sqrt((centerx_box2 - centerx_box1)**2 + (centery_box2 - centery_box1)**2)

    half_width_sum = ((x2_box1 - x1_box1) + (x2_box2 - x1_box2))/2
    half_height_sum = ((y1_box1 - y2_box1) + (y1_box2 - y2_box2))/2

    #print(center_distance, half_width_sum, half_height_sum)

    if half_width_sum >= center_distance:
        return True

    if half_height_sum >= center_distance:
        return True

    else :
        return False

json_file_name = 'context.json'    
data = {"example": "Context"}
with open(json_file_name, 'w') as json_file:
    json.dump(data, json_file)

count = 1
count2 = 1
c2=0
for file in os.listdir('out_csv'):
    if file in app:
        continue
    app.append(file)
    try:
        #print(file, count)
        
        count +=1
        count2 +=1
        if count==500:
            print('500 done')
            count=0

        #if count2 == 2500:
            #break
        df = pd.read_csv(f'out_csv/{file}')
        class_order = df[df['frames'] == 'frame0']['class'].tolist()
        df = df[df['class'].isin(class_order)].copy()
        df['y1'] = -df['y1']
        df['y2'] = -df['y2']
        df['x1'] -= 256
        df['y1'] += 256
        df['x2'] -= 256
        df['y2'] += 256

        con = df['frames'] == 'frame0'
        class_id = df.loc[con, 'class']
        nob = num(len(list(class_id)))

        s1=''
        #s1 = f'Initially there are {nob} objects, {num(len(list(class_id))-1)} static and one dynamic object. '
        s1 = f'Initially there are {nob} objects, {num(len(list(class_id))-1)} static and one dynamic object. The static objects are '

        frame_0 = df[df['frames'] == 'frame0']
        frame_0['center'] = frame_0.apply(lambda row: ((row['x1'] + row['x2']) / 2, (row['y1'] + row['y2']) / 2), axis=1)

        s2=''
        for index, row in frame_0.iterrows():
            current_center = row['center']
            df_without_current = frame_0[frame_0.index != index]  
            current_object = row['class']
            x1_0 = row['x1']
            y1_0 = row['y1']
            d = get_direction_origin(x1_0, y1_0)
            
            nearest_distance = float('inf')
            nearest_object = None
            nearest_center = None
            
            for _, other_row in df_without_current.iterrows():
                other_center = other_row['center']
                distance = calculate_distance(current_center, other_center)
                
                if distance < nearest_distance:
                    nearest_distance = distance
                    nearest_object = other_row['class']
                    nearest_center = other_center
                    x1_1 = other_row['x1']
                    y1_1 = other_row['y1']

            pos = get_direction2(x1_0, y1_0, x1_1, y1_1)

            #s2 = s2+ f'The {current_object} is located in the {d} direction from the origin and {pos} of {nearest_object}. '
            s2 = s2 + f'{current_object}, '

        frame_0 = df[df['frames'] == 'frame0']
        class_present = list(frame_0['class'])
        changes_det = False
        dynamic=[]
        for i in range(len(class_present)):
            frame_p = df[df['class'] == class_present[i]]
            frame_p = frame_p.reset_index(drop=True)
            frame_p1 = frame_p.drop(columns=['frames', 'class', 'x2', 'y2'])
            frame_p2 = frame_p.drop(columns=['frames', 'class', 'x1', 'y1'])
            for j in range(1, len(frame_p)):
                prev_frame1 = frame_p1.iloc[j-1]
                curr_frame1 = frame_p1.iloc[j]

                prev_frame2 = frame_p2.iloc[j-1]
                curr_frame2 = frame_p2.iloc[j]

                if not prev_frame1.equals(curr_frame1) and not prev_frame2.equals(curr_frame2):
                    changes_det = True
                    first_change = j
                    break
            
            if changes_det == True:
                dynamic.append((class_present[i], frame_p.iloc[first_change,0], j))

            changes_det = False


        sorted_dynamic = sorted(dynamic, key=lambda x:x[2])
        dynamic1 = sorted_dynamic[0]
        sorted_dynamic = sorted_dynamic[1:]

        for i in range(len(sorted_dynamic)):
            frame = sorted_dynamic[i][1]
            ob = sorted_dynamic[i][0]
            frame_n = df[df['frames'] == frame]
            frame_n = frame_n.reset_index(drop=True)
            row = frame_n[frame_n['class'] == ob]
            bbox_1 = row[['x1', 'y1', 'x2', 'y2']]
            ind = bbox_1.index
            ind = ind[0]
            for j in range(len(frame_n)):
                if j!=ind:
                    row = frame_n.iloc[j]
                    bbox_n = row[['x1', 'y1', 'x2', 'y2']]
                    res = is_colliding(bbox_1, bbox_n)
                    if res == True:
                        cl = list(row[['class']])
                        sorted_dynamic[i] = sorted_dynamic[i]+(cl[0],)

        frame_dy = df[df['frames'] == dynamic1[1]]
        row1 = frame_dy[frame_dy['class'] == dynamic1[0]]
        bbox1 = row1[['x1', 'y1', 'x2', 'y2']]
        f = dynamic1[2]
        frame_dy2 = df[df['frames'] == f'frame{f+3}']
        row2 = frame_dy2[frame_dy2['class'] == dynamic1[0]]
        bbox2 = row2[['x1', 'y1', 'x2', 'y2']]
        x1_0 = bbox1.iloc[0,0]
        y1_0 = bbox1.iloc[0,1]
        x1_1 = bbox2.iloc[0,0]
        y1_1 = bbox2.iloc[0,1]
        dy_pos = get_direction_origin(x1_0, y1_0)
        dy_dir = get_direction(x1_0, y1_0, x1_1, y1_1)

        for i in range(len(sorted_dynamic)):
            if len(sorted_dynamic[i]) >=4:
                collidee = sorted_dynamic[i][0]
                collider = sorted_dynamic[i][3]
                f = sorted_dynamic[i][2]
                #getting the direction of collider
                frames1 = df[df['frames'] == f'frame{f}']
                row1 = frames1[frames1['class'] == collider]
                bbox1 = row1[['x1', 'y1', 'x2', 'y2']]
                frames2 = df[df['frames'] == f'frame{f+1}']
                row2 = frames2[frames2['class'] == collider]
                bbox2 = row2[['x1', 'y1', 'x2', 'y2']]
                x1_0 = bbox1.iloc[0,0]
                y1_0 = bbox1.iloc[0,1]
                x1_1 = bbox2.iloc[0,0]
                y1_1 = bbox2.iloc[0,1]
                collider_dir = get_direction(x1_0, y1_0, x1_1, y1_1)

                #getting the direction of coliidee
                frames1 = df[df['frames'] == f'frame{f}']
                row1 = frames1[frames1['class'] == collidee]
                bbox1 = row1[['x1', 'y1', 'x2', 'y2']]
                frames2 = df[df['frames'] == f'frame{f+1}']
                row2 = frames2[frames2['class'] == collidee]
                bbox2 = row2[['x1', 'y1', 'x2', 'y2']]
                x2_0 = bbox1.iloc[0,0]
                y2_0 = bbox1.iloc[0,1]
                x2_1 = bbox2.iloc[0,0]
                y2_1 = bbox2.iloc[0,1]

                collidee_dir = get_direction(x2_0, y2_0, x2_1, y2_1)
                index=i
                break

        s3=''
        #s3 = f'The dynamic {dynamic1[0]} is located in the {dy_pos} with respect to the origin at the center. It is moving in the {dy_dir} direction and collides with the {sorted_dynamic[0][0]}. After the collision the {collider} moves in {collider_dir} and {collidee} moves in {collidee_dir} direction. '
        #s3 = f'The dynamic {dynamic1[0]} collides with the {sorted_dynamic[0][0]}. '
        s3 = f'The dynamic {dynamic1[0]} collides with the {sorted_dynamic[0][0]}.It is moving in the {dy_dir} direction and collides with the {sorted_dynamic[0][0]}. After the collision the {collider} moves in {collider_dir} and {collidee} moves in {collidee_dir} direction.  '
        s4=''
        for i in range(index+1, len(sorted_dynamic)):
            if len(sorted_dynamic[i]) >=4:
                collidee = sorted_dynamic[i][0]
                collider = sorted_dynamic[i][3]
                f = sorted_dynamic[i][2]

                #getting the direction of collider
                frames1 = df[df['frames'] == f'frame{f}']
                row1 = frames1[frames1['class'] == collider]
                bbox1 = row1[['x1', 'y1', 'x2', 'y2']]
                frames2 = df[df['frames'] == f'frame{f+1}']
                row2 = frames2[frames2['class'] == collider]
                bbox2 = row2[['x1', 'y1', 'x2', 'y2']]
                x1_0 = bbox1.iloc[0,0]
                y1_0 = bbox1.iloc[0,1]
                x1_1 = bbox2.iloc[0,0]
                y1_1 = bbox2.iloc[0,1]
                collider_dir = get_direction(x1_0, y1_0, x1_1, y1_1)

                #getting the direction of coliidee
                frames1 = df[df['frames'] == f'frame{f}']
                row1 = frames1[frames1['class'] == collidee]
                bbox1 = row1[['x1', 'y1', 'x2', 'y2']]
                frames2 = df[df['frames'] == f'frame{f+1}']
                row2 = frames2[frames2['class'] == collidee]
                bbox2 = row2[['x1', 'y1', 'x2', 'y2']]
                x1_0 = bbox1.iloc[0,0]
                y1_0 = bbox1.iloc[0,1]
                x1_1 = bbox2.iloc[0,0]
                y1_1 = bbox2.iloc[0,1]
                collidee_dir = get_direction(x1_0, y1_0, x1_1, y1_1)


                s4 = s4+f'The {collider} collides with the {collidee}. The {collider} moves in the {collider_dir} and the {collidee} moves in the {collidee_dir} direction. '
                #s4 = s4 + f'The {collider} collides with the {collidee}. '
        
        s=s1+s2+s3+s4
        n = file.split(".")[0]
        new_data = {f"{n}": s}
        #print(s)
        #break

        with open(json_file_name, 'r') as j_file:
            existing_data = json.load(j_file)

        existing_data.update(new_data)

        with open(json_file_name, 'w') as j_file:
            json.dump(existing_data, j_file, indent=4)

    except:
        c2 = c2+1
        csv_file_path = "fails.csv"
        fname=[file]
        with open(csv_file_path, mode='a', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(fname)
        pass

#print(c2)


In [1]:
import json

file1_path = 'relative_pos_context1.json'
file2_path = 'relative_pos_context2.json'

with open(file1_path, 'r') as file1:
    data1 = json.load(file1)


with open(file2_path, 'r') as file2:
    data2 = json.load(file2)


merged_data = {**data1, **data2}


merged_file_path = 'relative_pos_context.json'

with open(merged_file_path, 'w') as merged_file:
    json.dump(merged_data, merged_file, indent=4)

print(f'Merged JSON data saved to {merged_file_path}')


Merged JSON data saved to relative_pos_context.json


In [36]:
import json
f = open('IID/counterfactual_gt.json')
qa_data = json.load(f)
remove = qa_data['remove']
add = qa_data['add']
replace = qa_data['replace']
'''
print(len(remove))
print(len(add))
print(len(replace))
print(len(remove)+len(add)+len(replace))
'''

f2 = open('collision_directions_context.json')
con = json.load(f2)

articles=[]
c1= 0
c2 = 0
for rem in range(len(remove)):
    try:
        noc = len(remove[rem]['choices'])
        eg = remove[rem]['example_file']
        context = con[eg]
        for i in range(noc):
                question = remove[rem]['question']
                question = question.replace("What will happen, if", "If")
                ch = remove[rem]['choices'][i]['choice']
                ch = "will the " + ch + " ?" 
                question = question.replace("?", ch)
                ans = remove[rem]['choices'][i]['answer']
                inputs = {"context": context, "question": question, "answer": ans}
                articles.append(inputs)

    except:
        c1 +=1
        pass

data = pd.DataFrame(articles)
print(len(data))
#data.head()

data.to_csv('shreyas/collisions_directions/remove.csv')

articles2=[]
c1= 0
for rem in range(len(add)):
    try:
        noc = len(add[rem]['choices'])
        eg = add[rem]['example_file']
        context = con[eg]
        for i in range(noc):
                question = add[rem]['question']
                question = question.replace("What will happen, if", "If")
                ch = add[rem]['choices'][i]['choice']
                ch = "will the " + ch + " ?" 
                question = question.replace("?", ch)
                ans = add[rem]['choices'][i]['answer']
                inputs = {"context": context, "question": question, "answer": ans}
                articles.append(inputs)
                articles2.append(inputs)

    except:
        c1 +=1
        pass

data = pd.DataFrame(articles)
#print(len(data))
#data.head()

data2 = pd.DataFrame(articles2)
print(len(data2))
#data2.head()

data2.to_csv('shreyas/collisions_directions/plan_add.csv')

articles3=[]
c1= 0
for rem in range(len(replace)):
    try:
        noc = len(replace[rem]['choices'])
        eg = replace[rem]['example_file']
        context = con[eg]
        for i in range(noc):
                question = replace[rem]['question']
                question = question.replace("What will happen, if", "If")
                ch = replace[rem]['choices'][i]['choice']
                ch = "will the " + ch + " ?" 
                question = question.replace("?", ch)
                ans = replace[rem]['choices'][i]['answer']
                inputs = {"context": context, "question": question, "answer": ans}
                articles.append(inputs)
                articles3.append(inputs)

    except:
        c1 +=1
        pass

data = pd.DataFrame(articles)
#print(len(data))
#data.head()

data3 = pd.DataFrame(articles3)
print(len(data3))
#data3.head()

data3.to_csv('shreyas/collisions_directions/replace.csv')
data.to_csv('shreyas/collisions_directions/data.csv')

print(len(data))
data.head()


9133
25712
4840
39685


,context,question,answer
0,"Initially there are six objects, five static a...",If the teal cardboard sphere is removed will ...,false
1,"Initially there are six objects, five static a...",If the teal cardboard sphere is removed will ...,true
2,"Initially there are six objects, five static a...",If the teal cardboard sphere is removed will ...,true
3,"Initially there are six objects, five static a...",If the teal cardboard sphere is removed will ...,false
4,"Initially there are six objects, five static a...",If the teal cardboard sphere is removed will ...,false


In [37]:
f = open('IID/planning_gt.json')
plan_data = json.load(f)
plan_rem = plan_data['remove']
plan_rep = plan_data['replace']
plan_add = plan_data['add']
print(len(plan_rem))
print(len(plan_rep))
print(len(plan_add))
total = len(plan_add)+len(plan_rem)+len(plan_rep)
print(total)

f2 = open('collision_directions_context.json')
con = json.load(f2)
articles2 = []
for i in range(len(plan_rem)):
    ex_file = plan_rem[i]['example_file']
    #ex_file = ex_file.split('.')[0]
    if ex_file not in con:
        continue
    ques = plan_rem[i]['question']
    ans = plan_rem[i]['answers']
    context = con[ex_file]
    inputs = {'context' : context, 'question': ques, 'answer' : ans}
    articles2.append(inputs)

for i in range(len(plan_rep)):
    ex_file = plan_rep[i]['example_file']
    #ex_file = ex_file.split('.')[0]
    if ex_file not in con:
        continue
    ques = plan_rep[i]['question']
    ans = plan_rep[i]['answers']
    context = con[ex_file]
    inputs = {'context' : context, 'question': ques, 'answer' : ans}
    articles2.append(inputs)

for i in range(len(plan_add)):
    ex_file = plan_rem[i]['example_file']
    #ex_file = ex_file.split('.')[0]
    if ex_file not in con:
        continue
    ques = plan_add[i]['question']
    ans = plan_add[i]['answers']
    context = con[ex_file]
    inputs = {'context' : context, 'question': ques, 'answer' : ans}
    articles2.append(inputs)

data3 = pd.DataFrame(articles2)
print(len(data3))
data3.to_csv('shreyas/collisions_directions/plan_data.csv')

5137
1825
3115
10077
9548


In [38]:
f = open('IID/descriptive_gt.json')
des_data = json.load(f)
f2 = open('collision_directions_context.json')
con = json.load(f2)
articles = []
data = des_data['data']

for i in range(len(data)):
    ex_file = data[i]['example_file']
    ex_file = ex_file.split('.')[0]
    if ex_file not in con:
        continue
    ques = data[i]['question']
    ans = data[i]['answer']
    context = con[ex_file]
    inputs = {'context' : context, 'question': ques, 'answer' : ans}
    articles.append(inputs)

data2 = pd.DataFrame(articles)
print(len(data2))
data2.to_csv('shreyas/collisions_directions/des_data.csv')

34237


In [19]:
import pandas as pd
fp = 'dataset4/des_data.csv'
df = pd.read_csv(fp)
len(df)

34074

In [57]:
import csv
import pandas as pd
import math
import numpy as np
import os
import json

import warnings
warnings.filterwarnings("ignore")

def calculate_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

def num(v):
    if v == 1:
        return 'one'
    elif v == 2:
        return 'two'
    elif v == 3:
        return 'three'
    elif v == 4:
        return 'four'
    elif v == 5:
        return 'five'
    elif v == 6:
        return 'six'
    elif v == 7:
        return 'seven'
    elif v == 8:
        return 'eight'
    elif v == 9:
        return 'nine'
    

def get_direction_origin(x_a, y_a):
    direction = ""

    if x_a > 0 and y_a > 0:
        direction = "northeast"
    elif x_a > 0 and y_a < 0:
        direction = "southeast"
    elif x_a < 0 and y_a < 0:
        direction = "southwest"
    elif x_a < 0 and y_a > 0:
        direction = "northwest"
    elif x_a > 0:
        direction = "east"
    elif y_a > 0:
        direction = "north"
    elif x_a < 0:
        direction = "west"
    elif y_a < 0:
        direction = "south"

    return direction

def get_direction(x_a, y_a, x_b, y_b):
    x_diff = x_b - x_a
    y_diff = y_b - y_a
    directions = ""

    angle = math.degrees(math.atan2(y_diff, x_diff))  # Convert negative angles to positive
    if angle<0:
        angle = angle+360

    if angle>337.5 and angle<=22.5 and angle==0 and angle==360:
        directions = 'East'
    elif angle>22.5 and angle<=67.5:
        directions = 'Northeast'
    elif angle>67.5 and angle<=112.5:
        directions = 'North'
    elif angle>112.5 and angle<=157.5:
        directions = 'Northwest'
    elif angle>157.5 and angle<=202.5:
        directions = 'West'
    elif angle>202.5 and angle<=247.5:
        directions = 'Southwest'
    elif angle>247.5 and angle<=292.5:
        directions= 'South'
    elif angle>292.5 and angle<=337.5:
        directions = 'Southeast'

    return directions

def get_direction2(x_a, y_a, x_b, y_b):
    x_diff = x_b - x_a
    y_diff = y_b - y_a
    directions = ""

    angle = math.degrees(math.atan2(y_diff, x_diff))  # Convert negative angles to positive
    if angle<0:
        angle = angle+360

    if angle>337.5 and angle<=22.5 and angle==0 and angle==360:
        directions = 'West'
    elif angle>22.5 and angle<=67.5:
        directions = 'Southwest'
    elif angle>67.5 and angle<=112.5:
        directions = 'South'
    elif angle>112.5 and angle<=157.5:
        directions = 'Southeast'
    elif angle>157.5 and angle<=202.5:
        directions = 'East'
    elif angle>202.5 and angle<=247.5:
        directions = 'Northeast'
    elif angle>247.5 and angle<=292.5:
        directions= 'North'
    elif angle>292.5 and angle<=337.5:
        directions = 'Northwest'

    return directions

def is_colliding(box1, box2):

    x1_box1 = int(box1['x1'].iloc[0])
    y1_box1 = int(box1['y1'].iloc[0])
    x2_box1 = int(box1['x2'].iloc[0])
    y2_box1 = int(box1['y2'].iloc[0])

    x1_box2 = int(box2['x1'])
    y1_box2 = int(box2['y1'])
    x2_box2 = int(box2['x2'])
    y2_box2 = int(box2['y2'])

    centerx_box1 = int((x1_box1 + x2_box1)/2)
    centery_box1 = int((y1_box1 + y2_box1)/2)

    centerx_box2 = int((x1_box2 + x2_box2)/2)
    centery_box2 = int((y1_box2 + y2_box2)/2)

    center_distance = math.sqrt((centerx_box2 - centerx_box1)**2 + (centery_box2 - centery_box1)**2)

    half_width_sum = ((x2_box1 - x1_box1) + (x2_box2 - x1_box2))/2
    half_height_sum = ((y1_box1 - y2_box1) + (y1_box2 - y2_box2))/2

    #print(center_distance, half_width_sum, half_height_sum)

    if half_width_sum >= center_distance:
        return True

    if half_height_sum >= center_distance:
        return True

    else :
        return False

df = pd.read_csv('out_csv/example_36.csv')
class_order = df[df['frames'] == 'frame0']['class'].tolist()
df = df[df['class'].isin(class_order)].copy()
df['y1'] = -df['y1']
df['y2'] = -df['y2']
df['x1'] -= 256
df['y1'] += 256
df['x2'] -= 256
df['y2'] += 256

con = df['frames'] == 'frame0'
class_id = df.loc[con, 'class']
nob = num(len(list(class_id)))

s1=''
s1 = f'Initially there are {nob} objects, {num(len(list(class_id))-1)} static and one dynamic object. '

frame_0 = df[df['frames'] == 'frame0']
frame_0['center'] = frame_0.apply(lambda row: ((row['x1'] + row['x2']) / 2, (row['y1'] + row['y2']) / 2), axis=1)

s2=''
for index, row in frame_0.iterrows():
    current_center = row['center']
    df_without_current = frame_0[frame_0.index != index]  
    current_object = row['class']
    x1_0 = row['x1']
    y1_0 = row['y1']
    d = get_direction_origin(x1_0, y1_0)
    
    nearest_distance = float('inf')
    nearest_object = None
    nearest_center = None
    
    for _, other_row in df_without_current.iterrows():
        other_center = other_row['center']
        distance = calculate_distance(current_center, other_center)
        
        if distance < nearest_distance:
            nearest_distance = distance
            nearest_object = other_row['class']
            nearest_center = other_center
            x1_1 = other_row['x1']
            y1_1 = other_row['y1']

    pos = get_direction2(x1_0, y1_0, x1_1, y1_1)

    s2 = s2+ f'The {current_object} is located in the {d} direction from the origin and {pos} of {nearest_object}. '

frame_0 = df[df['frames'] == 'frame0']
class_present = list(frame_0['class'])
changes_det = False
dynamic=[]
for i in range(len(class_present)):
    frame_p = df[df['class'] == class_present[i]]
    frame_p = frame_p.reset_index(drop=True)
    frame_p1 = frame_p.drop(columns=['frames', 'class', 'x2', 'y2'])
    frame_p2 = frame_p.drop(columns=['frames', 'class', 'x1', 'y1'])
    for j in range(1, len(frame_p)):
        prev_frame1 = frame_p1.iloc[j-1]
        curr_frame1 = frame_p1.iloc[j]

        prev_frame2 = frame_p2.iloc[j-1]
        curr_frame2 = frame_p2.iloc[j]

        if not prev_frame1.equals(curr_frame1) and not prev_frame2.equals(curr_frame2):
            changes_det = True
            first_change = j
            break
    
    if changes_det == True:
        dynamic.append((class_present[i], frame_p.iloc[first_change,0], j))

    changes_det = False


sorted_dynamic = sorted(dynamic, key=lambda x:x[2])
dynamic1 = sorted_dynamic[0]
sorted_dynamic = sorted_dynamic[1:]

for i in range(len(sorted_dynamic)):
    frame = sorted_dynamic[i][1]
    ob = sorted_dynamic[i][0]
    frame_n = df[df['frames'] == frame]
    frame_n = frame_n.reset_index(drop=True)
    row = frame_n[frame_n['class'] == ob]
    bbox_1 = row[['x1', 'y1', 'x2', 'y2']]
    ind = bbox_1.index
    ind = ind[0]
    for j in range(len(frame_n)):
        if j!=ind:
            row = frame_n.iloc[j]
            bbox_n = row[['x1', 'y1', 'x2', 'y2']]
            res = is_colliding(bbox_1, bbox_n)
            if res == True:
                cl = list(row[['class']])
                sorted_dynamic[i] = sorted_dynamic[i]+(cl[0],)

frame_dy = df[df['frames'] == dynamic1[1]]
row1 = frame_dy[frame_dy['class'] == dynamic1[0]]
bbox1 = row1[['x1', 'y1', 'x2', 'y2']]
f = dynamic1[2]
frame_dy2 = df[df['frames'] == f'frame{f+2}']
row2 = frame_dy2[frame_dy2['class'] == dynamic1[0]]
bbox2 = row2[['x1', 'y1', 'x2', 'y2']]
x1_0 = bbox1.iloc[0,0]
y1_0 = bbox1.iloc[0,1]
x1_1 = bbox2.iloc[0,0]
y1_1 = bbox2.iloc[0,1]
dy_pos = get_direction_origin(x1_0, y1_0)
dy_dir = get_direction(x1_0, y1_0, x1_1, y1_1)

for i in range(len(sorted_dynamic)):
    if len(sorted_dynamic[i]) >=4:
        collidee = sorted_dynamic[i][0]
        collider = sorted_dynamic[i][3]
        f = sorted_dynamic[i][2]
        #getting the direction of collider
        frames1 = df[df['frames'] == f'frame{f}']
        row1 = frames1[frames1['class'] == collider]
        bbox1 = row1[['x1', 'y1', 'x2', 'y2']]
        frames2 = df[df['frames'] == f'frame{f+1}']
        row2 = frames2[frames2['class'] == collider]
        bbox2 = row2[['x1', 'y1', 'x2', 'y2']]
        x1_0 = bbox1.iloc[0,0]
        y1_0 = bbox1.iloc[0,1]
        x1_1 = bbox2.iloc[0,0]
        y1_1 = bbox2.iloc[0,1]
        collider_dir = get_direction(x1_0, y1_0, x1_1, y1_1)

        #getting the direction of coliidee
        frames1 = df[df['frames'] == f'frame{f}']
        row1 = frames1[frames1['class'] == collidee]
        bbox1 = row1[['x1', 'y1', 'x2', 'y2']]
        frames2 = df[df['frames'] == f'frame{f+1}']
        row2 = frames2[frames2['class'] == collidee]
        bbox2 = row2[['x1', 'y1', 'x2', 'y2']]
        x2_0 = bbox1.iloc[0,0]
        y2_0 = bbox1.iloc[0,1]
        x2_1 = bbox2.iloc[0,0]
        y2_1 = bbox2.iloc[0,1]

        collidee_dir = get_direction(x2_0, y2_0, x2_1, y2_1)
        index=i
        break

s3=''
s3 = f'The dynamic {dynamic1[0]} is located in the {dy_pos} with respect to the origin at the center. It is moving in the {dy_dir} direction and collides with the {sorted_dynamic[0][0]}. After the collision the {collider} moves in {collider_dir} and {collidee} moves in {collidee_dir} direction. '
s4=''

for i in range(index+1, len(sorted_dynamic)):
    try:
        if len(sorted_dynamic[i]) >=4:
            collidee = sorted_dynamic[i][0]
            collider = sorted_dynamic[i][3]
            f = sorted_dynamic[i][2]

            #getting the direction of collider
            frames1 = df[df['frames'] == f'frame{f}']
            row1 = frames1[frames1['class'] == collider]
            bbox1 = row1[['x1', 'y1', 'x2', 'y2']]
            frames2 = df[df['frames'] == f'frame{f+1}']
            row2 = frames2[frames2['class'] == collider]
            bbox2 = row2[['x1', 'y1', 'x2', 'y2']]
            x1_0 = bbox1.iloc[0,0]
            y1_0 = bbox1.iloc[0,1]
            x1_1 = bbox2.iloc[0,0]
            y1_1 = bbox2.iloc[0,1]
            collider_dir = get_direction(x1_0, y1_0, x1_1, y1_1)

            #getting the direction of coliidee
            frames1 = df[df['frames'] == f'frame{f}']
            row1 = frames1[frames1['class'] == collidee]
            bbox1 = row1[['x1', 'y1', 'x2', 'y2']]
            frames2 = df[df['frames'] == f'frame{f+1}']
            row2 = frames2[frames2['class'] == collidee]
            bbox2 = row2[['x1', 'y1', 'x2', 'y2']]
            x1_0 = bbox1.iloc[0,0]
            y1_0 = bbox1.iloc[0,1]
            x1_1 = bbox2.iloc[0,0]
            y1_1 = bbox2.iloc[0,1]
            collidee_dir = get_direction(x1_0, y1_0, x1_1, y1_1)


        s4 = s4+f' The {collider} collides with the {collidee}. The {collider} moves in the {collider_dir} and the {collidee} moves in the {collidee_dir}.'
    except:
        sent = s4.split(".")
        l= len(sent)
        s4 = ".".join(sent[:(l-3)])

s=s1+s2+s3+s4
print(s)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [8]:
import csv
import pandas as pd
import math
import numpy as np
import os
import json
import warnings
warnings.filterwarnings("ignore")

def calculate_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

def num(v):
    if v == 1:
        return 'one'
    elif v == 2:
        return 'two'
    elif v == 3:
        return 'three'
    elif v == 4:
        return 'four'
    elif v == 5:
        return 'five'
    elif v == 6:
        return 'six'
    elif v == 7:
        return 'seven'
    elif v == 8:
        return 'eight'
    elif v == 9:
        return 'nine'
    

def get_direction_origin(x_a, y_a):
    direction = ""

    if x_a > 0 and y_a > 0:
        direction = "northeast"
    elif x_a > 0 and y_a < 0:
        direction = "southeast"
    elif x_a < 0 and y_a < 0:
        direction = "southwest"
    elif x_a < 0 and y_a > 0:
        direction = "northwest"
    elif x_a > 0:
        direction = "east"
    elif y_a > 0:
        direction = "north"
    elif x_a < 0:
        direction = "west"
    elif y_a < 0:
        direction = "south"

    return direction

def get_direction(x_a, y_a, x_b, y_b):
    x_diff = x_b - x_a
    y_diff = y_b - y_a
    directions = ""

    angle = math.degrees(math.atan2(y_diff, x_diff))  # Convert negative angles to positive
    if angle<0:
        angle = angle+360

    if angle>337.5 and angle<=22.5 and angle==0 and angle==360:
        directions = 'East'
    elif angle>22.5 and angle<=67.5:
        directions = 'Northeast'
    elif angle>67.5 and angle<=112.5:
        directions = 'North'
    elif angle>112.5 and angle<=157.5:
        directions = 'Northwest'
    elif angle>157.5 and angle<=202.5:
        directions = 'West'
    elif angle>202.5 and angle<=247.5:
        directions = 'Southwest'
    elif angle>247.5 and angle<=292.5:
        directions= 'South'
    elif angle>292.5 and angle<=337.5:
        directions = 'Southeast'

    return directions

def get_direction2(x_a, y_a, x_b, y_b):
    x_diff = x_b - x_a
    y_diff = y_b - y_a
    directions = ""

    angle = math.degrees(math.atan2(y_diff, x_diff))  # Convert negative angles to positive
    if angle<0:
        angle = angle+360

    if angle>337.5 and angle<=22.5 and angle==0 and angle==360:
        directions = 'West'
    elif angle>22.5 and angle<=67.5:
        directions = 'Southwest'
    elif angle>67.5 and angle<=112.5:
        directions = 'South'
    elif angle>112.5 and angle<=157.5:
        directions = 'Southeast'
    elif angle>157.5 and angle<=202.5:
        directions = 'East'
    elif angle>202.5 and angle<=247.5:
        directions = 'Northeast'
    elif angle>247.5 and angle<=292.5:
        directions= 'North'
    elif angle>292.5 and angle<=337.5:
        directions = 'Northwest'

    return directions

def is_colliding(box1, box2):

    x1_box1 = int(box1['x1'].iloc[0])
    y1_box1 = int(box1['y1'].iloc[0])
    x2_box1 = int(box1['x2'].iloc[0])
    y2_box1 = int(box1['y2'].iloc[0])

    x1_box2 = int(box2['x1'])
    y1_box2 = int(box2['y1'])
    x2_box2 = int(box2['x2'])
    y2_box2 = int(box2['y2'])

    centerx_box1 = int((x1_box1 + x2_box1)/2)
    centery_box1 = int((y1_box1 + y2_box1)/2)

    centerx_box2 = int((x1_box2 + x2_box2)/2)
    centery_box2 = int((y1_box2 + y2_box2)/2)

    center_distance = math.sqrt((centerx_box2 - centerx_box1)**2 + (centery_box2 - centery_box1)**2)

    half_width_sum = ((x2_box1 - x1_box1) + (x2_box2 - x1_box2))/2
    half_height_sum = ((y1_box1 - y2_box1) + (y1_box2 - y2_box2))/2

    #print(center_distance, half_width_sum, half_height_sum)

    if half_width_sum >= center_distance:
        return True

    if half_height_sum >= center_distance:
        return True

    else :
        return False
    
json_file_name = 'only_collision_context.json'
  
data = {"example": "Context"}
with open(json_file_name, 'w') as json_file:    #change the context file
    json.dump(data, json_file)

count = 1
c2=0

# CSV file path
csv_file_path = "fails/fails4.csv"

# Open the CSV file and extract names
with open(csv_file_path, 'r') as csv_file:
    files = csv_file.read().splitlines()

for file in files:
    try:
        #print(file, count)
        count +=1
        df = pd.read_csv(f'out_csv/{file}')
        class_order = df[df['frames'] == 'frame0']['class'].tolist()
        df = df[df['class'].isin(class_order)].copy()
        df['y1'] = -df['y1']
        df['y2'] = -df['y2']
        df['x1'] -= 256
        df['y1'] += 256
        df['x2'] -= 256
        df['y2'] += 256

        con = df['frames'] == 'frame0'
        class_id = df.loc[con, 'class']
        nob = num(len(list(class_id)))

        s1=''
        s1 = f'Initially there are {nob} objects, {num(len(list(class_id))-1)} static and one dynamic object. '

        frame_0 = df[df['frames'] == 'frame0']
        frame_0['center'] = frame_0.apply(lambda row: ((row['x1'] + row['x2']) / 2, (row['y1'] + row['y2']) / 2), axis=1)

        s2=''
        for index, row in frame_0.iterrows():
            current_center = row['center']
            df_without_current = frame_0[frame_0.index != index]  
            current_object = row['class']
            x1_0 = row['x1']
            y1_0 = row['y1']
            d = get_direction_origin(x1_0, y1_0)
            
            nearest_distance = float('inf')
            nearest_object = None
            nearest_center = None
            
            for _, other_row in df_without_current.iterrows():
                other_center = other_row['center']
                distance = calculate_distance(current_center, other_center)
                
                if distance < nearest_distance:
                    nearest_distance = distance
                    nearest_object = other_row['class']
                    nearest_center = other_center
                    x1_1 = other_row['x1']
                    y1_1 = other_row['y1']

            pos = get_direction2(x1_0, y1_0, x1_1, y1_1)

            s2 = s2+ f'The {current_object} is located in the {d} direction from the origin and {pos} of {nearest_object}. '

        frame_0 = df[df['frames'] == 'frame0']
        class_present = list(frame_0['class'])
        changes_det = False
        dynamic=[]
        for i in range(len(class_present)):
            frame_p = df[df['class'] == class_present[i]]
            frame_p = frame_p.reset_index(drop=True)
            frame_p1 = frame_p.drop(columns=['frames', 'class', 'x2', 'y2'])
            frame_p2 = frame_p.drop(columns=['frames', 'class', 'x1', 'y1'])
            for j in range(1, len(frame_p)):
                prev_frame1 = frame_p1.iloc[j-1]
                curr_frame1 = frame_p1.iloc[j]

                prev_frame2 = frame_p2.iloc[j-1]
                curr_frame2 = frame_p2.iloc[j]

                if not prev_frame1.equals(curr_frame1) and not prev_frame2.equals(curr_frame2):
                    changes_det = True
                    first_change = j
                    break
            
            if changes_det == True:
                dynamic.append((class_present[i], frame_p.iloc[first_change,0], j))

            changes_det = False


        sorted_dynamic = sorted(dynamic, key=lambda x:x[2])
        dynamic1 = sorted_dynamic[0]
        sorted_dynamic = sorted_dynamic[1:]

        for i in range(len(sorted_dynamic)):
            frame = sorted_dynamic[i][1]
            ob = sorted_dynamic[i][0]
            frame_n = df[df['frames'] == frame]
            frame_n = frame_n.reset_index(drop=True)
            row = frame_n[frame_n['class'] == ob]
            bbox_1 = row[['x1', 'y1', 'x2', 'y2']]
            ind = bbox_1.index
            ind = ind[0]
            for j in range(len(frame_n)):
                if j!=ind:
                    row = frame_n.iloc[j]
                    bbox_n = row[['x1', 'y1', 'x2', 'y2']]
                    res = is_colliding(bbox_1, bbox_n)
                    if res == True:
                        cl = list(row[['class']])
                        sorted_dynamic[i] = sorted_dynamic[i]+(cl[0],)

        frame_dy = df[df['frames'] == dynamic1[1]]
        row1 = frame_dy[frame_dy['class'] == dynamic1[0]]
        bbox1 = row1[['x1', 'y1', 'x2', 'y2']]
        f = dynamic1[2]
        frame_dy2 = df[df['frames'] == f'frame{f+2}']
        row2 = frame_dy2[frame_dy2['class'] == dynamic1[0]]
        bbox2 = row2[['x1', 'y1', 'x2', 'y2']]
        x1_0 = bbox1.iloc[0,0]
        y1_0 = bbox1.iloc[0,1]
        x1_1 = bbox2.iloc[0,0]
        y1_1 = bbox2.iloc[0,1]
        dy_pos = get_direction_origin(x1_0, y1_0)
        dy_dir = get_direction(x1_0, y1_0, x1_1, y1_1)

        for i in range(len(sorted_dynamic)):
            if len(sorted_dynamic[i]) >=4:
                collidee = sorted_dynamic[i][0]
                collider = sorted_dynamic[i][3]
                f = sorted_dynamic[i][2]
                #getting the direction of collider
                frames1 = df[df['frames'] == f'frame{f}']
                row1 = frames1[frames1['class'] == collider]
                bbox1 = row1[['x1', 'y1', 'x2', 'y2']]
                frames2 = df[df['frames'] == f'frame{f+1}']
                row2 = frames2[frames2['class'] == collider]
                bbox2 = row2[['x1', 'y1', 'x2', 'y2']]
                x1_0 = bbox1.iloc[0,0]
                y1_0 = bbox1.iloc[0,1]
                x1_1 = bbox2.iloc[0,0]
                y1_1 = bbox2.iloc[0,1]
                collider_dir = get_direction(x1_0, y1_0, x1_1, y1_1)

                #getting the direction of coliidee
                frames1 = df[df['frames'] == f'frame{f}']
                row1 = frames1[frames1['class'] == collidee]
                bbox1 = row1[['x1', 'y1', 'x2', 'y2']]
                frames2 = df[df['frames'] == f'frame{f+1}']
                row2 = frames2[frames2['class'] == collidee]
                bbox2 = row2[['x1', 'y1', 'x2', 'y2']]
                x2_0 = bbox1.iloc[0,0]
                y2_0 = bbox1.iloc[0,1]
                x2_1 = bbox2.iloc[0,0]
                y2_1 = bbox2.iloc[0,1]

                collidee_dir = get_direction(x2_0, y2_0, x2_1, y2_1)
                index=i
                break

        s3=''
        s3 = f'The dynamic {dynamic1[0]} is located in the {dy_pos} with respect to the origin at the center. It is moving in the {dy_dir} direction and collides with the {sorted_dynamic[0][0]}. After the collision the {collider} moves in {collider_dir} and {collidee} moves in {collidee_dir} direction. '
        s4=''
        for i in range(index+1, len(sorted_dynamic)):
            try:
                if len(sorted_dynamic[i]) >=4:
                    collidee = sorted_dynamic[i][0]
                    collider = sorted_dynamic[i][3]
                    f = sorted_dynamic[i][2]

                    #getting the direction of collider
                    frames1 = df[df['frames'] == f'frame{f}']
                    row1 = frames1[frames1['class'] == collider]
                    bbox1 = row1[['x1', 'y1', 'x2', 'y2']]
                    frames2 = df[df['frames'] == f'frame{f+1}']
                    row2 = frames2[frames2['class'] == collider]
                    bbox2 = row2[['x1', 'y1', 'x2', 'y2']]
                    x1_0 = bbox1.iloc[0,0]
                    y1_0 = bbox1.iloc[0,1]
                    x1_1 = bbox2.iloc[0,0]
                    y1_1 = bbox2.iloc[0,1]
                    collider_dir = get_direction(x1_0, y1_0, x1_1, y1_1)

                    #getting the direction of coliidee
                    frames1 = df[df['frames'] == f'frame{f}']
                    row1 = frames1[frames1['class'] == collidee]
                    bbox1 = row1[['x1', 'y1', 'x2', 'y2']]
                    frames2 = df[df['frames'] == f'frame{f+1}']
                    row2 = frames2[frames2['class'] == collidee]
                    bbox2 = row2[['x1', 'y1', 'x2', 'y2']]
                    x1_0 = bbox1.iloc[0,0]
                    y1_0 = bbox1.iloc[0,1]
                    x1_1 = bbox2.iloc[0,0]
                    y1_1 = bbox2.iloc[0,1]
                    collidee_dir = get_direction(x1_0, y1_0, x1_1, y1_1)


                    s4 = s4+f'The {collider} collides with the {collidee}. The {collider} moves in the {collider_dir} and the {collidee} moves in the {collidee_dir}. '

            except:
                sent = s4.split(".")
                l= len(sent)
                s4 = ".".join(sent[:(l-2)])
                
        s=s1+s2+s3+s4
        print(s)
        n = file.split(".")[0]
        new_data = {f"{n}": s}

        with open(json_file_name, 'r') as j_file:
            existing_data = json.load(j_file)

        existing_data.update(new_data)

        with open(json_file_name, 'w') as j_file:
            json.dump(existing_data, j_file, indent=4)

    except:
        c2 = c2+1
        csv_file_path = "fails/fails4.csv"
        fname=[file]
        with open(csv_file_path, mode='a', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(fname)
        pass

#print(c2)


In [67]:
import json
f = open('IID/counterfactual_gt.json')
qa_data = json.load(f)
remove = qa_data['remove']
add = qa_data['add']
replace = qa_data['replace']
print(len(remove))
print(len(add_json))
print(len(replace))
print(len(remove)+len(add_json)+len(replace))

3314
8535
1723
13572


In [60]:
f2 = open('context2.json')
con = json.load(f2)

In [62]:
articles=[]
c1= 0
c2 = 0
for rem in range(len(remove)):
    try:
        noc = len(remove[rem]['choices'])
        eg = remove[rem]['example_file']
        context = con[eg]
        for i in range(noc):
                question = remove[rem]['question']
                question = question.replace("What will happen, if", "If")
                ch = remove[rem]['choices'][i]['choice']
                ch = "will the " + ch + " ?" 
                question = question.replace("?", ch)
                ans = remove[rem]['choices'][i]['answer']
                inputs = {"context": context, "question": question, "answer": ans}
                articles.append(inputs)

    except:
        c1 +=1
        pass

In [63]:
data = pd.DataFrame(articles)
print(len(data))
data.head()

9481


,context,question,answer
0,"Initially there are six objects, five static a...",If the teal cardboard sphere is removed will ...,true
1,"Initially there are six objects, five static a...",If the teal cardboard sphere is removed will ...,false
2,"Initially there are six objects, five static a...",If the teal cardboard sphere is removed will ...,true
3,"Initially there are six objects, five static a...",If the teal cardboard sphere is removed will ...,false
4,"Initially there are six objects, five static a...",If the teal cardboard sphere is removed will ...,false


In [64]:
data.to_csv('remove.csv')

In [65]:
c1

40

In [68]:
articles2=[]
c1= 0
for rem in range(len(add)):
    try:
        noc = len(add[rem]['choices'])
        eg = add[rem]['example_file']
        context = con[eg]
        for i in range(noc):
                question = add[rem]['question']
                question = question.replace("What will happen, if", "If")
                ch = add[rem]['choices'][i]['choice']
                ch = "will the " + ch + " ?" 
                question = question.replace("?", ch)
                ans = add[rem]['choices'][i]['answer']
                inputs = {"context": context, "question": question, "answer": ans}
                articles.append(inputs)
                articles2.append(inputs)

    except:
        c1 +=1
        pass

In [69]:
data = pd.DataFrame(articles)
print(len(data))
data.head()

36223


,context,question,answer
0,"Initially there are six objects, five static a...",If the teal cardboard sphere is removed will ...,true
1,"Initially there are six objects, five static a...",If the teal cardboard sphere is removed will ...,false
2,"Initially there are six objects, five static a...",If the teal cardboard sphere is removed will ...,true
3,"Initially there are six objects, five static a...",If the teal cardboard sphere is removed will ...,false
4,"Initially there are six objects, five static a...",If the teal cardboard sphere is removed will ...,false


In [70]:
data2 = pd.DataFrame(articles2)
print(len(data2))
data2.head()

26742


,context,question,answer
0,"Initially there are five objects, four static ...",If the purple cardboard cube is added to the r...,false
1,"Initially there are five objects, four static ...",If the purple cardboard cube is added to the r...,false
2,"Initially there are five objects, four static ...",If the purple cardboard cube is added to the r...,true
3,"Initially there are five objects, four static ...",If the purple cardboard cube is added to the r...,true
4,"Initially there are five objects, four static ...",If the purple cardboard cube is added to the r...,false


In [71]:
data2.to_csv('dataset3/add.csv')

In [72]:
c1

95

In [73]:
articles3=[]
c1= 0
for rem in range(len(replace)):
    try:
        noc = len(replace[rem]['choices'])
        eg = replace[rem]['example_file']
        context = con[eg]
        for i in range(noc):
                question = replace[rem]['question']
                question = question.replace("What will happen, if", "If")
                ch = replace[rem]['choices'][i]['choice']
                ch = "will the " + ch + " ?" 
                question = question.replace("?", ch)
                ans = replace[rem]['choices'][i]['answer']
                inputs = {"context": context, "question": question, "answer": ans}
                articles.append(inputs)
                articles3.append(inputs)

    except:
        c1 +=1
        pass

In [74]:
data = pd.DataFrame(articles)
print(len(data))
data.head()

41295


,context,question,answer
0,"Initially there are six objects, five static a...",If the teal cardboard sphere is removed will ...,true
1,"Initially there are six objects, five static a...",If the teal cardboard sphere is removed will ...,false
2,"Initially there are six objects, five static a...",If the teal cardboard sphere is removed will ...,true
3,"Initially there are six objects, five static a...",If the teal cardboard sphere is removed will ...,false
4,"Initially there are six objects, five static a...",If the teal cardboard sphere is removed will ...,false


In [75]:
data3 = pd.DataFrame(articles3)
print(len(data3))
data3.head()

5072


,context,question,answer
0,"Initially there are six objects, five static a...",If the teal cardboard sphere is replaced by t...,false
1,"Initially there are six objects, five static a...",If the teal cardboard sphere is replaced by t...,true
2,"Initially there are six objects, five static a...",If the olive aluminium sphere is replaced by ...,false
3,"Initially there are six objects, five static a...",If the olive aluminium sphere is replaced by ...,true
4,"Initially there are six objects, five static a...",If the olive aluminium sphere is replaced by ...,false


In [76]:
c1

23

In [77]:
data3.to_csv('dataset3/replace.csv')

In [ ]:
data.to_csv('dataset3/data.csv')